In [2]:
var ds = require('dstools');
var input = require('../examples/data/3features/input-m.json'); 
var diversibute = require('../index');

#### Wie schneiden die beiden Algorithmen bei 100x Tests ab?  

Gruppengröße: 26  
Gruppenanzahl: 3

In [10]:
var groups = 5;

In [11]:
// 100x runs of monte carlo
var scoresMonteCarlo = [];
var resultsMonteCarlo = {};

// Initialize empty promise collection
var runs = [];

for(let run = 0; run < 50; run++) {
    
    // Calculate groups with monte carlo
    runs.push(diversibute.monteCarlo(input, groups).then(result => {
                
        // Collect result and score
        resultsMonteCarlo[result.score] = result;
        scoresMonteCarlo.push(result.score);
        
        return;
    }));
}

var highscoreMonteCarlo = undefined;

// Run async tasks in a sequence
runs.reduce((promiseChain, currentTask) => {    
    return promiseChain.then(chainResults =>        
             currentTask.then(currentResult =>            
                  [ ...chainResults, currentResult ]        
             )    
        );
    },Promise.resolve([])).then(arrayOfResults => {    

        // Do something with all results
        console.log(scoresMonteCarlo);

        // Calculate highest score of this batch
        highscoreMonteCarlo = Math.max(...scoresMonteCarlo);

        console.log(highscoreMonteCarlo);
    });

[
  21.973638186555604,   26.1282288514046,  45.24326647573308,
   37.59287231531427,  30.66717964323027, 36.082420549722684,
   42.34404496023048, 26.855037532280036,  44.27576585485039,
   32.06081368915922, 47.074441495648955, 22.197270250281946,
   50.47745110383772, 43.128050337251224,  34.89423807392524,
   35.71122253179824, 46.417508162425776,  36.97380077011019,
  30.435639389603338,  44.87473951765567,  43.51626037123455,
   21.18071380850232,  50.06500407651998, 47.031779775563365,
   59.81901522829877,  37.67567151284936, 12.425120561704887,
   41.96807963191284, 19.622204365574248,  37.68112099767312,
   33.37862820017192,  23.96488986281304, 28.022036707030864,
   36.45592709368607,  47.43087596331114, 53.833600823037486,
   26.94505746266448,  32.71864140529544, 21.240370193266827,
  40.216571642840606, 55.627472860294375, 25.806810377361415,
   53.12856231131279,  37.25525124788406,  37.72642785390505,
   51.99142453909873, 30.662270582115568,  32.32393055537494,
   39.

In [12]:
// 100x runs of genetic
var scoresGenetic = [];
var resultsGenetic = {};
var runs = [];

// Run algorithm x times
for(let run = 0; run < 50; run++) {
    runs.push(diversibute.genetic(input, groups, {populationStartSize: 20, populationMaxSize: 100, evolutions: 1000}).then(result => {
        
        // Collect result and score
        resultsGenetic[result.score] = result;
        return scoresGenetic.push(result.score);
    }));
}

var highscoreGenetic = undefined;

// We must wait until all calculations are done
Promise.all(runs).then(values => {
    
    console.log(scoresGenetic);

    // Calculate highest score of this batch
    highscoreGenetic = Math.max(...scoresGenetic);

    console.log(highscoreGenetic);
});

[
  79.60652258851074, 74.07697448758226, 73.97089936566469,
  75.17844345854284,  73.8235332655006, 80.32534331132936,
  79.90157687026816, 80.78200431797002, 69.85506609461359,
  79.70249591248891, 72.06576545849227, 82.78752482673403,
  81.23458800837787, 79.69889343534464,  81.1102385490328,
  77.24056873318624,  77.9835795162515, 79.16834522172468,
   77.2413705183193, 78.96165550563032, 75.28894133091093,
   78.9649501608937, 74.04447359555463,  76.4455367860298,
  72.13607625058513, 80.24434783760405, 78.77377681379285,
  73.97726858893746, 77.08063641426776, 77.58908146958036,
   76.9906182907458, 76.47223777817068, 77.21538557476057,
  77.19972932474762, 81.81884431933915,   78.859219378982,
  78.36438830890731, 76.63899147577918, 75.32709296888954,
  80.63910126636364, 78.71560816117929,  78.3179448337145,
  75.83287036733742, 77.60081961566061, 77.13292967781476,
  79.28011428616847, 71.17373441801607, 78.90471474078092,
  76.30343200379812, 77.21733401412543
]
82.7875248267

## Vergleich Algorithmen

In [13]:
// Calculate overall hitrate

// Highest score
var highestScore = (highscoreMonteCarlo > highscoreGenetic) ? highscoreMonteCarlo : highscoreGenetic;

var hitRateMonteCarlo = scoresMonteCarlo.map(v => v/highestScore);
var hitRateGenetic = scoresGenetic.map(v => v/highestScore);

In [14]:
var trace1 = {
  x: hitRateMonteCarlo,
  type: "histogram",
  opacity: 0.5,
  marker: {
     color: 'green',
  },
  name: 'Monte Carlo'
};
var trace2 = {
  x: hitRateGenetic,
  type: "histogram",
  opacity: 0.6,
  marker: {
     color: 'red',
  },
  name: 'Genetic Algorithm'
};

var data = [trace1, trace2];
var layout = {barmode: "overlay", xaxis: {title: 'Trefferquote'}, yaxis: {title: 'Häufigkeit'}};

ds.Collection().plotly(data, layout).show();

## Darstellung "bestes" Ergebnis

In [15]:
// Retrieve winner combination
var winnerSet = (highestScore == highscoreMonteCarlo) ? resultsMonteCarlo : resultsGenetic;
var winner = winnerSet[highestScore];

// Generate empty traces
/*var traces = ['rgba(217, 217, 217, 0.14)', 'rgba(217, 217, 217, 0.14)', 'rgba(217, 217, 217, 0.14)'].map(c => {
    return {x: [], y: [], z: [],
        mode: 'markers',
        marker: {
            size: 12,
            line: {
            color: c,
            width: 0.5},
            opacity: 0.8},
        type: 'scatter3d'};
});
*/
var traces = [];
for(let i = 0; i < groups; i++) {
    traces.push({x: [], y: [], z: [],
        mode: 'markers',
        marker: {
            size: 12,
            line: {
                color: 'rgba(217, 217, 217, 0.14)',
                width: 0.5
            },
            opacity: 0.8
        },
        type: 'scatter3d'
    });
}

// Loop input data and build group traces
Object.keys(input).forEach((member, index) => {
    
    // Retrieve member group 
    var group = winner.combination[index];
    
    traces[group-1].x.push(input[member][0]);
    traces[group-1].y.push(input[member][1]);
    traces[group-1].z.push(input[member][2]);    
});

// Print 3D Model with group traces and colors
ds.Collection().plotly(traces,{margin: {l: 0, r: 0, b: 0, t: 0 }}).show();